In [5]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import numpy as np
import cv2
import os
from PIL import Image
import random
from tqdm import tqdm  # Import tqdm for progress bar


In [6]:

class DDOSDataset(Dataset):
    def __init__(self, root_dir=r'DS2\DDOS\data', split='train', transform=None, max_images=30000):
        self.root_dir = os.path.join(root_dir, split)
        self.transform = transform
        self.max_images = max_images  # Limit to 500 images
        self.image_paths = []
        self.mask_paths = []

        # Collect all image and mask paths
        environments = ['neighbourhood', 'park']
        for env in environments:
            env_dir = os.path.join(self.root_dir, env)
            if not os.path.exists(env_dir):
                continue
            flight_folders = sorted([f for f in os.listdir(env_dir) if f.isdigit()], key=int)
            for flight in flight_folders:
                flight_dir = os.path.join(env_dir, flight)
                image_dir = os.path.join(flight_dir, 'image')
                seg_dir = os.path.join(flight_dir, 'segmentation')
                if not (os.path.exists(image_dir) and os.path.exists(seg_dir)):
                    continue
                images = sorted(os.listdir(image_dir))
                masks = sorted(os.listdir(seg_dir))
                for img, mask in zip(images, masks):
                    self.image_paths.append(os.path.join(image_dir, img))
                    self.mask_paths.append(os.path.join(seg_dir, mask))

        # Randomly select 500 images (or fewer if total < 500)
        if len(self.image_paths) > self.max_images:
            indices = random.sample(range(len(self.image_paths)), self.max_images)
            self.image_paths = [self.image_paths[i] for i in indices]
            self.mask_paths = [self.mask_paths[i] for i in indices]

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        # Load RGB image
        img = Image.open(self.image_paths[idx]).convert('RGB')
        img = np.array(img, dtype=np.float32) / 255.0

        # Load segmentation mask
        mask = Image.open(self.mask_paths[idx])
        mask = np.array(mask, dtype=np.uint8)

        if self.transform:
            img = self.transform(img)
            mask = self.transform(mask)

        img = cv2.resize(img, (128, 128))
        mask = cv2.resize(mask, (128, 128), interpolation=cv2.INTER_NEAREST)

        # Class mapping with default value
        class_mapping = {255: 0, 240: 1, 225: 2, 210: 3, 195: 4, 180: 5, 165: 6, 150: 7, 140: 8, 0: 9}
        mask = np.vectorize(lambda x: class_mapping.get(x, 9))(mask)

        img = torch.tensor(img.transpose(2, 0, 1), dtype=torch.float32)
        mask = torch.tensor(mask, dtype=torch.long)

        return img, mask

In [7]:

class LiquidNeuralNetwork(nn.Module):
    def __init__(self, input_channels=3, hidden_size=128, num_classes=10):
        super(LiquidNeuralNetwork, self).__init__()
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        self.tau = 1.0  # Time constant

        self.input_layer = nn.Conv2d(input_channels, hidden_size, kernel_size=3, padding=1)
        self.liquid_layer = nn.Conv2d(hidden_size, hidden_size, kernel_size=3, padding=1)
        self.output_layer = nn.Conv2d(hidden_size, num_classes, kernel_size=1)

    def forward(self, x, hidden=None):
        if hidden is None:
            hidden = torch.zeros(x.size(0), self.hidden_size, x.size(2), x.size(3)).to(x.device)

        input_state = torch.tanh(self.input_layer(x))
        hidden = (1 - 1/self.tau) * hidden + (1/self.tau) * torch.tanh(self.liquid_layer(input_state + hidden))
        output = self.output_layer(hidden)
        return output, hidden
    # def ode_func(self, hidden, input_state):
    #     return (-hidden / self.tau) + (1 / self.tau) * torch.tanh(self.liquid_layer(input_state + hidden))
    # def forward(self, x, hidden=None):
    #     if hidden is None:
    #         hidden = torch.zeros(x.size(0), self.hidden_size, x.size(2), x.size(3)).to(x.device)

    #     input_state = torch.tanh(self.input_layer(x))

    #     # RK4 solver
    #     dt = 1.0
    #     k1 = self.ode_func(hidden, input_state)
    #     k2 = self.ode_func(hidden + 0.5 * dt * k1, input_state)
    #     k3 = self.ode_func(hidden + 0.5 * dt * k2, input_state)
    #     k4 = self.ode_func(hidden + dt * k3, input_state)
    #     hidden = hidden + (dt / 6.0) * (k1 + 2 * k2 + 2 * k3 + k4)

    #     output = self.output_layer(hidden)
    #     return output, hidden

In [8]:
dataset = DDOSDataset(split='train', max_images=30000)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)
print(f"Dataset size: {len(dataset)} images")


Dataset size: 30000 images


In [9]:
model = LiquidNeuralNetwork(input_channels=3, hidden_size=128, num_classes=10)
print("model")
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)
model

model


LiquidNeuralNetwork(
  (input_layer): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (liquid_layer): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (output_layer): Conv2d(128, 10, kernel_size=(1, 1), stride=(1, 1))
)

In [10]:


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print("optimizer")
criterion = nn.CrossEntropyLoss()
num_epochs = 20  # Short test



for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    # Wrap dataloader with tqdm for progress bar
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)
    
    for images, masks in progress_bar:
        images, masks = images.to(device), masks.to(device)
        hidden = None
        outputs, hidden = model(images, hidden)
        loss = criterion(outputs, masks)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Update progress bar with current loss
        progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})
    
    # Print final epoch loss (optional, since tqdm shows progress)
    avg_loss = running_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


optimizer


Epoch 1/20: 100%|█████████████████████████████████████████| 1875/1875 [58:15<00:00,  1.86s/it, loss=0.4346]


Epoch 1/20, Loss: 0.4222


Epoch 2/20: 100%|█████████████████████████████████████████| 1875/1875 [58:15<00:00,  1.86s/it, loss=0.3042]


Epoch 2/20, Loss: 0.3544


...

Epoch 18/20, Loss: 0.2725


Epoch 19/20: 100%|████████████████████████████████████████| 1875/1875 [56:48<00:00,  1.82s/it, loss=0.3047]


Epoch 19/20, Loss: 0.2716


Epoch 20/20: 100%|████████████████████████████████████████| 1875/1875 [56:14<00:00,  1.80s/it, loss=0.2655]

Epoch 20/20, Loss: 0.2710


In [12]:
model_path = 'liquid_nn_model_final.pth'
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to liquid_nn_model_final.pth


In [14]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
loaded_model = LiquidNeuralNetwork(input_channels=3, hidden_size=128, num_classes=10).to(device)
loaded_model.load_state_dict(torch.load(model_path))
loaded_model.eval()

LiquidNeuralNetwork(
  (input_layer): Conv2d(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (liquid_layer): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (output_layer): Conv2d(128, 10, kernel_size=(1, 1), stride=(1, 1))
)

In [ ]:

import time
start_time = time.time()
val_dataset = DDOSDataset(split='validation')
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=True, num_workers=0, pin_memory=True)
with torch.no_grad():
    for images, masks in val_loader:
        images, masks = images.to(device), masks.to(device)
        torch.cuda.synchronize()
        t0 = time.time()
        outputs, _ = loaded_model(images)
        torch.cuda.synchronize()
        preds = torch.argmax(outputs, dim=1)
        print(f"Batch inference time: {time.time() - t0:.4f} seconds")

        # Visualization for first batch
        img = images[0].cpu().numpy().transpose(1, 2, 0)
        mask = masks[0].cpu().numpy()
        pred = preds[0].cpu().numpy()

        display_size = (512, 512)
        img_resized = cv2.resize(img, display_size, interpolation=cv2.INTER_LINEAR)
        mask_resized = cv2.resize(mask, display_size, interpolation=cv2.INTER_NEAREST)
        pred_resized = cv2.resize(pred, display_size, interpolation=cv2.INTER_NEAREST)

        img_resized = cv2.cvtColor(img_resized, cv2.COLOR_RGB2BGR)

        def colorize_mask(mask, num_classes=10):
            colors = [
                [0, 0, 0], [255, 0, 0], [0, 255, 0], [0, 0, 255], [255, 255, 0],
                [255, 0, 255], [0, 255, 255], [128, 128, 128], [255, 128, 0], [128, 0, 128]
            ]
            colored_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
            for i in range(num_classes):
                colored_mask[mask == i] = colors[i]
            return colored_mask

        mask_colored = colorize_mask(mask_resized)
        pred_colored = colorize_mask(pred_resized)

        cv2.namedWindow('RGB', cv2.WINDOW_NORMAL)
        cv2.namedWindow('Ground Truth', cv2.WINDOW_NORMAL)
        cv2.namedWindow('Prediction', cv2.WINDOW_NORMAL)

        cv2.imshow('RGB', img_resized)
        cv2.imshow('Ground Truth', mask_colored)
        cv2.imshow('Prediction', pred_colored)

        cv2.waitKey(0)
        break

print(f"Total validation time: {time.time() - start_time:.4f} seconds")
cv2.destroyAllWindows()

Batch inference time: 0.3026 seconds


In [14]:


optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
print("optimizer")
criterion = nn.CrossEntropyLoss()
num_epochs = 20  # Short test



for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    
    # Wrap dataloader with tqdm for progress bar
    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{num_epochs}", leave=True)
    
    for images, masks in progress_bar:
        images, masks = images.to(device), masks.to(device)
        hidden = None
        outputs, hidden = model(images, hidden)
        loss = criterion(outputs, masks)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
        # Update progress bar with current loss
        progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})
    
    # Print final epoch loss (optional, since tqdm shows progress)
    avg_loss = running_loss / len(dataloader)
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {avg_loss:.4f}")


optimizer


Epoch 1/20: 100%|███████████████████████████████████████████████████████| 125/125 [00:28<00:00,  4.35it/s, loss=0.4844]


Epoch 1/20, Loss: 0.5640


Epoch 2/20: 100%|███████████████████████████████████████████████████████| 125/125 [00:21<00:00,  5.68it/s, loss=0.6728]


Epoch 2/20, Loss: 0.5072


...

Epoch 18/20, Loss: 0.4005


Epoch 19/20: 100%|██████████████████████████████████████████████████████| 125/125 [00:24<00:00,  5.03it/s, loss=0.4333]


Epoch 19/20, Loss: 0.3983


Epoch 20/20: 100%|██████████████████████████████████████████████████████| 125/125 [00:23<00:00,  5.27it/s, loss=0.3354]

Epoch 20/20, Loss: 0.3964


In [ ]:
val_dataset = DDOSDataset(split='validation')
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=True)

model.eval()
with torch.no_grad():
    for images, masks in val_loader:
        images, masks = images.to(device), masks.to(device)
        outputs, _ = model(images)
        preds = torch.argmax(outputs, dim=1)

        img = images[0].cpu().numpy().transpose(1, 2, 0)
        mask = masks[0].cpu().numpy()
        pred = preds[0].cpu().numpy()

        cv2.imshow('RGB', img)
        cv2.imshow('Ground Truth', mask.astype(np.float32) / 9.0)  # Scale 0-9 to 0-1
        cv2.imshow('Prediction', pred.astype(np.float32) / 9.0)
        cv2.waitKey(0)
        break
cv2.destroyAllWindows()

In [6]:
print(torch.cuda.is_available())  # Should return True if GPU is set up
print(torch.version.cuda)  # Shows CUDA version PyTorch is using

True
11.7


In [18]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import cv2

# Assuming DDOSDataset and model are already defined
val_dataset = DDOSDataset(split='test')  # Your dataset class
val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# Assuming device and model are defined earlier
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = LiquidNeuralNetwork(...).to(device)

model.eval()
with torch.no_grad():
    for images, masks in val_loader:
        images, masks = images.to(device), masks.to(device)
        outputs, _ = model(images)
        preds = torch.argmax(outputs, dim=1)

        # Convert tensors to numpy
        img = images[0].cpu().numpy().transpose(1, 2, 0)  # [H, W, C]
        mask = masks[0].cpu().numpy()  # [H, W]
        pred = preds[0].cpu().numpy()  # [H, W]

        # Resize images for better visibility (e.g., to 512x512)
        display_size = (512, 512)
        img_resized = cv2.resize(img, display_size, interpolation=cv2.INTER_LINEAR)
        mask_resized = cv2.resize(mask, display_size, interpolation=cv2.INTER_NEAREST)
        pred_resized = cv2.resize(pred, display_size, interpolation=cv2.INTER_NEAREST)

        # Convert RGB to BGR for OpenCV
        img_resized = cv2.cvtColor(img_resized, cv2.COLOR_RGB2BGR)

        # Apply color mapping to masks for better visualization
        def colorize_mask(mask, num_classes=10):
            # Create a color map (BGR format for OpenCV)
            colors = [
                [0, 0, 0],    # 0: Black
                [255, 0, 0],  # 1: Blue
                [0, 255, 0],  # 2: Green
                [0, 0, 255],  # 3: Red
                [255, 255, 0],# 4: Yellow
                [255, 0, 255],# 5: Magenta
                [0, 255, 255],# 6: Cyan
                [128, 128, 128],# 7: Gray
                [255, 128, 0],# 8: Orange
                [128, 100, 128] # 9: Purple
            ]
            colored_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
            for i in range(num_classes):
                colored_mask[mask == i] = colors[i]
            return colored_mask

        mask_colored = colorize_mask(mask_resized)
        pred_colored = colorize_mask(pred_resized)

        # Create resizable windows
        cv2.namedWindow('RGB', cv2.WINDOW_NORMAL)
        cv2.namedWindow('Ground Truth', cv2.WINDOW_NORMAL)
        cv2.namedWindow('Prediction', cv2.WINDOW_NORMAL)

        # Display images
        cv2.imshow('RGB', img_resized)
        cv2.imshow('Ground Truth', mask_colored)
        cv2.imshow('Prediction', pred_colored)

        # Wait for key press and allow resizing
        cv2.waitKey(0)
        break

cv2.destroyAllWindows()

In [1]:
import torch
print(torch.__version__)          # Should show "2.0.1+cu117"
print(torch.cuda.is_available())  # Should be True
print(torch.version.cuda)         # Should show "11.7"
print(torch.cuda.current_device())# Should show 0 (or your GPU index)
print(torch.cuda.get_device_name(0))  # Should show your GPU name, e.g., "NVIDIA GeForce RTX 3060"

2.0.1+cu117
True
11.7
0
NVIDIA GeForce RTX 3080 Laptop GPU


In [1]:
import torch
from torch.utils.data import DataLoader
import numpy as np
import cv2
import time

# Device setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Assuming DDOSDataset and LiquidNeuralNetwork are defined
val_dataset = DDOSDataset(split='validation')
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

model = model.to(device)

# Time the validation
model.eval()
start_time = time.time()
with torch.no_grad():
    for images, masks in val_loader:
        images, masks = images.to(device), masks.to(device)
        torch.cuda.synchronize()  # Sync GPU
        t0 = time.time()
        outputs, _ = model(images)
        torch.cuda.synchronize()  # Sync GPU
        preds = torch.argmax(outputs, dim=1)
        print(f"Batch inference time: {time.time() - t0:.4f} seconds")

        # Visualization (optional, comment out for pure timing)
        img = images[0].cpu().numpy().transpose(1, 2, 0)
        mask = masks[0].cpu().numpy()
        pred = preds[0].cpu().numpy()

        display_size = (512, 512)
        img_resized = cv2.resize(img, display_size, interpolation=cv2.INTER_LINEAR)
        mask_resized = cv2.resize(mask, display_size, interpolation=cv2.INTER_NEAREST)
        pred_resized = cv2.resize(pred, display_size, interpolation=cv2.INTER_NEAREST)

        img_resized = cv2.cvtColor(img_resized, cv2.COLOR_RGB2BGR)

        def colorize_mask(mask, num_classes=10):
            colors = [
                [0, 0, 0], [255, 0, 0], [0, 255, 0], [0, 0, 255], [255, 255, 0],
                [255, 0, 255], [0, 255, 255], [128, 128, 128], [255, 128, 0], [128, 0, 128]
            ]
            colored_mask = np.zeros((mask.shape[0], mask.shape[1], 3), dtype=np.uint8)
            for i in range(num_classes):
                colored_mask[mask == i] = colors[i]
            return colored_mask

        mask_colored = colorize_mask(mask_resized)
        pred_colored = colorize_mask(pred_resized)

        cv2.namedWindow('RGB', cv2.WINDOW_NORMAL)
        cv2.namedWindow('Ground Truth', cv2.WINDOW_NORMAL)
        cv2.namedWindow('Prediction', cv2.WINDOW_NORMAL)

        cv2.imshow('RGB', img_resized)
        cv2.imshow('Ground Truth', mask_colored)
        cv2.imshow('Prediction', pred_colored)

        cv2.waitKey(0)
        break

print(f"Total validation time: {time.time() - start_time:.4f} seconds")
cv2.destroyAllWindows()

Using device: cuda


NameError: name 'DDOSDataset' is not defined

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

dataset = DDOSDataset(split='train', max_images=500)
dataloader = DataLoader(dataset, batch_size=4, shuffle=True)
model = LiquidNeuralNetwork(input_channels=3, hidden_size=128, num_classes=10).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, masks in dataloader:
        images, masks = images.to(device), masks.to(device)
        hidden = None
        outputs, hidden = model(images, hidden)
        loss = criterion(outputs, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(dataloader):.4f}")